In [4]:
import tensorflow as tensorflow
import matplotlib.pyplot as plt
import matplotlib.image as img
import pandas as pd
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split
from tensorflow import keras
import numpy as np
from tensorflow.keras import mixed_precision
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory
import os, os.path
import csv
import tensorflow_hub as hub
from tqdm import tqdm

batch_size = 32
img_size = 224
# Load the savedModel
model = tf.keras.models.load_model('Best_points.h5')


In [14]:
def tta_prediction(model, batch_size, img_size):
    test_location = "dataset/test_images"
    aug_gens = ImageDataGenerator(
            featurewise_center=False,
            samplewise_center=False,
            featurewise_std_normalization=False,
            samplewise_std_normalization=False,
            zca_whitening=False,
            rotation_range=10,
            shear_range=0.25,
            zoom_range=0.1,
            width_shift_range=0.1,
            height_shift_range=0.1,
            horizontal_flip=True,
            preprocessing_function=keras.applications.mobilenet_v2.preprocess_input,
    )



    tta_steps = 10
    predictions = []
    for i in tqdm(range(tta_steps)):
        preds = model.predict(aug_gens.flow_from_directory(
            directory=test_location,
            target_size=(img_size, img_size),
            batch_size=batch_size,
            classes=['.'],
            shuffle=False,
        ),
                              workers=16)
        predictions.append(preds)

    final_pred = np.mean(predictions, axis=0)
    return final_pred

def generate_augmented_images(batch_size, img_size):

    aug_gens = ImageDataGenerator(
            featurewise_center=False,
            samplewise_center=False,
            featurewise_std_normalization=False,
            samplewise_std_normalization=False,
            zca_whitening=False,
            validation_split=0.1,
            rotation_range=10,
            shear_range=0.25,
            zoom_range=0.1,
            width_shift_range=0.1,
            height_shift_range=0.1,
            horizontal_flip=True,
            vertical_flip=True,
            preprocessing_function=keras.applications.mobilenet_v2.preprocess_input,
    )

    train_data = aug_gens.flow_from_directory("dataset/train_images",
                                              subset="training",
                                              seed=1447,
                                              target_size=(img_size, img_size),
                                              batch_size=batch_size,
                                              class_mode="categorical")

    val_data = aug_gens.flow_from_directory("dataset/train_images",
                                            subset="validation",
                                            seed=1447,
                                            target_size=(img_size, img_size),
                                            batch_size=batch_size,
                                            class_mode="categorical")

    return train_data, val_data


def generate_augmented_test(batch_size, img_size):
    test_location = "dataset/test_images"
    test_data = ImageDataGenerator(
        preprocessing_function=keras.applications.mobilenet_v2.preprocess_input).flow_from_directory(
            directory=test_location,
            target_size=(img_size, img_size),
            batch_size=batch_size,
            classes=['.'],
            shuffle=False,
        )
    return test_data

In [19]:
## Matching Predictions with Correct Image ID
train_data, _ = generate_augmented_images(
    batch_size, img_size)
pred = tta_prediction(model, batch_size, img_size)
y_predict_max = np.argmax(pred, axis=1)

inverse_map = {v: k for k, v in train_data.class_indices.items()}

predictions = [inverse_map[k] for k in y_predict_max]
test_data = generate_augmented_test(batch_size, img_size)
files = test_data.filenames

results = pd.DataFrame({"image_id": files, "label": predictions})
results.image_id = results.image_id.apply(lambda x: x[2:])
results.to_csv("submission.csv", index=False)
results.head()

Found 9371 images belonging to 10 classes.
Found 1036 images belonging to 10 classes.


  0%|          | 0/10 [00:00<?, ?it/s]

Found 3469 images belonging to 1 classes.
109/109 [==============================] - 5s 45ms/step


 10%|█         | 1/10 [00:05<00:52,  5.86s/it]

Found 3469 images belonging to 1 classes.
109/109 [==============================] - 6s 47ms/step


...

 90%|█████████ | 9/10 [00:50<00:05,  5.46s/it]

Found 3469 images belonging to 1 classes.
109/109 [==============================] - 5s 42ms/step


100%|██████████| 10/10 [00:55<00:00,  5.59s/it]

Found 3469 images belonging to 1 classes.


,image_id,label
0,200001.jpg,hispa
1,200002.jpg,normal
2,200003.jpg,blast
3,200004.jpg,blast
4,200005.jpg,blast
